In [ ]:
pip install datasets peft evaluate transformers[torch] bitsandbytes trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 109.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
pip install accelerate -U

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig

In [ ]:
dataset = load_dataset("b-mc2/sql-create-context")

dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'context', 'answer'],
        num_rows: 78577
    })
})

In [ ]:
# dataset['train'][0]

In [ ]:
model_checkpoint = "stabilityai/StableBeluga-7B"
# Initialize the tokenizer and model
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, max_length=512)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
model.config.use_cache = False

In [ ]:
model.config.quantization_config.to_dict()

In [ ]:
lora_target_modules = [
    "query_key_value",
    "dense",
    "dense_h_to_4h",
    "dense_4h_to_h",
]
config = LoraConfig(
    r=16,  # attention heads
    lora_alpha=12,  # alpha scaling
    target_modules=lora_target_modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
import random

split_ratio = 0.8
eval_ratio = 0.2

# the 30% subset
total_examples = len(dataset["train"])
subset_size = int(total_examples * 0.2)
train_size = int(subset_size * split_ratio)
eval_size = subset_size - train_size
shuffled_indices = list(range(total_examples))
random.shuffle(shuffled_indices)
training_set = dataset["train"].select(shuffled_indices[:train_size])
evaluation_set = dataset["train"].select(
    shuffled_indices[train_size: train_size + eval_size]
)
split_dataset = DatasetDict({"train": training_set, "eval": evaluation_set})
split_dataset

In [ ]:
evaluation_set

In [ ]:
# hyperparameters
lr = 1e-4
batch_size = 4
num_epochs = 1
training_args = TrainingArguments(
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=lr,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=num_epochs,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir="outputs",
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=12,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["eval"],
    peft_config=config,
    dataset_text_field="question",
    max_seq_length=4096,
    tokenizer=tokenizer,
    args=training_args,
)

In [ ]:
# train model
trainer.train()

In [ ]:
model.push_to_hub("samadpls/querypls-prompt2sql")
tokenizer.push_to_hub("samadpls/querypls-prompt2sql")

In [ ]:
# DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'